### Define configuration files

In [ ]:
from box.sim_box import SimulationBox
from potential.force import Force
from cell.cell import Cell
from substrate.substrates import Substrate
from helper_functions import helper_functions as hf
from visuals.figure import Figure
from polarity import polarity

import matplotlib.pyplot as plt
import glob
import pandas as pd
import numpy as np
import os
import time
from importlib import reload

reload(polarity)

from cell import cell

reload(cell)
from cell.cell import Cell


im_kwargs = {"origin": "lower", "extent": [0, 50, 0, 50]}
cntr_kwargs = {
    "origin": "lower",
    "extent": [0, 50, 0, 50],
    "colors": ["black"],
    "linewidths": [2],
    "levels": [0.5],
}

### Some random functions

In [ ]:
def _masked(arr, masker):
    arr_masked = np.ones(arr.shape) * np.nan
    i, j = np.where(masker >= 0.5)
    arr_masked[i, j] = arr[i, j]
    return arr_masked


def plot_probs(cell, p1, p2, p, c, time, p_ratio):
    cntr = cell.contour[0][:, ::-1]

    if not isinstance(p1, np.ndarray):
        p1 = np.array([p1] * cntr.shape[0])

    if not isinstance(p2, np.ndarray):
        p2 = np.array([p2] * cntr.shape[0])

    plt.figure(figsize=(6, 6))
    plt.subplot(221)
    plt.title("filopodia")
    plt.scatter(cntr[:, 0], cntr[:, 1], c=p1, cmap="coolwarm")
    plt.colorbar()

    plt.subplot(222)
    plt.title("feedback")
    plt.scatter(cntr[:, 0], cntr[:, 1], c=p2, cmap="coolwarm")
    plt.colorbar()
    plt.scatter(*c, color="black", s=20)

    plt.subplot(223)
    plt.title("total")
    plt.scatter(cntr[:, 0], cntr[:, 1], c=p, cmap="coolwarm")
    plt.colorbar()

    plt.subplot(224)
    plt.title("Tension")
    plt.plot(time, p_ratio)
    plt.xlabel("Time")
    plt.ylabel(r"$P/P_0$")

    plt.subplots_adjust(wspace=1, hspace=0.7)
    plt.show()


### Initialize system

In [ ]:
def _build_system(simbox, cell_config, cell_rng_seed):
    # unpack
    N_mesh, L_box = simbox.N_mesh, simbox.L_box

    # define base substrate
    sub_config = simbox.sub_config
    xi = sub_config["xi"]
    kind = sub_config["kind"]
    sub = Substrate(N_mesh, L_box, xi)
    if kind == "two-state":
        chi = sub.two_state_sub(bridge_width=17)
    elif kind == "rectangular":
        chi = sub.rectangular()
    else:
        raise ValueError(f"{kind} for substrate is not understood.")

    # initialize cells
    cell = Cell(cell_config, simbox, cell_rng_seed)

    return cell, chi


def _define_paths(gid):
    SIMBOX_CONFIG = f"../configs/IM/grid_id{gid}/simbox.yaml"
    ENERGY_CONFIG = "../configs/energy.yaml"

    CELL_CONFIG = f"../configs/IM/grid_id{gid}/cell.yaml"

    return dict(
        simbox=SIMBOX_CONFIG,
        energy=ENERGY_CONFIG,
        cell=CELL_CONFIG,
    )


In [ ]:
# define various paths
gid = 8
paths = _define_paths(gid)

# initialize the simulation box
simbox = SimulationBox(paths["simbox"])
seed = int(time.time())
cell, chi = _build_system(simbox, paths["cell"], cell_rng_seed=seed)

# initialize the force calculator
force_calculator = Force(paths["energy"])

### Evolve the cell

In [ ]:
from IPython import display
import os

t = []
p_ratio = []
save_path = f"../output/IM/grid_id{gid}/run_0/visuals/"

N = 120000
cap = 250
R_ten = 1.25 * cell.R_eq

if not os.path.exists(save_path):
    os.makedirs(save_path)

for n in range(N):
    # needed more than once
    grad_x, grad_y, _ = hf.compute_gradients(cell.phi, cell.simbox.dx)
    grad_phi = np.array([grad_x, grad_y])

    if n % cap == 0:
        Figure.view_pol_field(cell, chi, dpi=100, path=save_path + f"img_{n}.png")
        print(
            f"Tension activated: {polarity.shape_induced_stress(cell, perim_0=2 * np.pi * R_ten):.2f}"
        )

    # polarization field (n+1)
    p_field_next = polarity.update_field(cell, grad_phi, chi, n)

    # phi_(n+1)
    phi_i_next, dF_dphi = hf._update_field(cell, grad_phi, force_calculator)

    # compute motility forces at time n
    fx_motil, fy_motil = force_calculator.cyto_motility_force(cell, grad_phi, chi)

    # compute thermodynamic forces at time n
    fx_thermo = dF_dphi * grad_x
    fy_thermo = dF_dphi * grad_y

    # UPDATE class variables now
    cell.phi = phi_i_next
    cell.p_field = p_field_next
    cell.contour = hf.find_contour(
        cell.phi, interpolate=cell.pol_model_kwargs["interpolate_cntrs"]
    )
    cell.cm = hf.compute_CM(cell)
    cell.v_cm = hf.compute_v_CM(cell)
    cell.vx = (fx_thermo + fx_motil) / cell.eta
    cell.vy = (fy_thermo + fy_motil) / cell.eta
